<a href="https://colab.research.google.com/github/chaerui7967/stock_predict_news_and_youtube/blob/master/youtube_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


## 휴장일 처리

In [3]:
def set_data(df,com_num = 0):  # panda, numpy, datetime, FinanceDataReader, konlpy, Counter
    # 일, 시간 나누기
    df['date'] = df['date'].astype(str)
    df['date_d'] = df['date'].str[:-2]
    df['date_h'] = df['date'].str[-2:]
    # 타입을 데이트 타입으로 만듬
    df['date_d'] = pd.to_datetime(df['date_d'])
    
    df = df.sort_values(by='date_d') # 일기준으로 오름차순 정렬
    
    # 널값 제거
    df = df.dropna()
    
    # 주말 및 공휴일 데이터
    ### Holidays
    try:
        # 서버가 열려있을 때
        db = pymysql.connect(user='root',
                             passwd='1234',
                             host='3.35.70.166',
                             db='proj',
                             charset='utf8')

        cursor = db.cursor(pymysql.cursors.DictCursor)

        sql = "select * from holidays"
        cursor.execute(sql)
        result = cursor.fetchall()
    
        # DataFrame으로 변경
        holi = pd.DataFrame(result)
        # db 닫기 --> 안하면 메모리 잡아먹음
        db.close()
    except:
        # 서버 없을 때 깃허브에서 바로 가져옴
        db_holi = 'https://raw.githubusercontent.com/chaerui7967/stock_predict_news_and_youtube/master/Sentiment_Analysis/data/holidays.csv'
        holi = pd.read_csv(db_holi)
    
    # date 컬럼을 날짜 형식으로 변경
    holi['date'] = pd.to_datetime(holi['date'])
    
    
    ### ===================주말 및 공휴일 제외
    
    ## 주말 및 공휴일만 추출
    market_closed = holi[holi['holiday']=="O"].reset_index(drop=True)
    
    ## 휴장일 List 생성
    market_closed_list = list(market_closed['date'])
    
    # 주말 및 공휴일 제외
    while len(df[df['date_d'].isin(market_closed_list)]['date_d']) !=0:
        for date in df[df['date_d'].isin(market_closed_list)]['date_d'].index:
            df['date_d'][date] += datetime.timedelta(1)
    
    return df

In [ ]:
youtube_df = pd.read_csv('/content/drive/My Drive/Final PJT - 업빛투/감성분석/data/youtube_total.csv')

In [ ]:
youtube_df = set_data(youtube_df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
youtube_df.drop(['date','length', 'url', 'date_h','description'], 1, inplace=True)

In [ ]:
youtube_df = youtube_df[['st_n', 'st_cd', 'ch_nm', 'date_d', 'title', 'text', 'views']]

In [ ]:
youtube_df.columns = ['st_n', 'st_cd', 'ch_nm', 'date', 'title', 'text', 'views']

In [ ]:
youtube_df.head()

,st_n,st_cd,ch_nm,date,title,text,views
1362,삼성전자,5930,한국경제TV,2012-06-26,[마감시황] 삼성전자 `흔들` 코스피도 `휘청`‥1820선 하락마감,매우 늘 증시 마감 상황 정리 해보겠습니다 증권 팀 지수 이 기자나와있습니다네 오늘...,30
60,하이닉스,660,한국경제TV,2012-07-02,"SK하이닉스, M12라인 가동 차세대 메모리 본격 강화",sk 하이닉스가 차세대 메모리 반도체 사업 확장에 본격적으로 돌입했습니다청주에 새로...,154
2016,LG화학,51910,한국경제TV,2012-07-14,6인의 마감전략 성공투자 오후증시2012년 07월 13일 방송,tweet4 7월 13일 금 일 장 마감 42분 전 이군요 자 6명의 전문가는어떤 ...,115
1283,삼성전자,5930,한국경제TV,2012-07-30,삼성전자 또 사상최대 실적 2분기 영업이익 6조7천억원,소식입니다 삼성전자가 또다시 사상 최대 실적을 경신했습니다 갤럭시 시리즈의 위력을 ...,36
1259,삼성전자,5930,한국경제TV,2012-09-13,"삼성전자, 반도체 중국생산 시대 연다",삼성전자가 중국 산시성 시안 시에서 낸드플래시 반도체 공장 기공식을열었습니다삼성전자...,164


In [ ]:
youtube_df.to_csv('./youtube_preprocessing.csv', index=False)

## 토큰화

In [4]:
!pip install konlpy 

     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 448 kB 38.6 MB/s 
     |████████████████████████████████| 86 kB 5.6 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [9]:
from konlpy.tag import Okt
okt = Okt()

In [5]:
youtube_df = pd.read_csv('/content/drive/My Drive/Final PJT - 업빛투/감성분석/data/youtube_preprocessing.csv')

In [11]:
# 불용어
url = 'https://raw.githubusercontent.com/chaerui7967/stock_predict_news_and_youtube/master/Sentiment_Analysis/data/stopwords_ver1.txt'
stopwords = list(pd.read_csv(url, header=None)[0])

In [14]:
# 데이터 전처리

youtube_df.text = youtube_df.text.str.replace('[으*]','')
youtube_df.text = youtube_df.text.str.replace('[\[\]]','')
youtube_df.text = youtube_df.text.str.replace('음악','')
youtube_df.text = youtube_df.text.str.replace('자','')
youtube_df.text = youtube_df.text.str.replace('[a-zA-Z]','')
youtube_df.text = youtube_df.text.str.replace('[구독*]','')
youtube_df.text = youtube_df.text.str.replace('[좋아요]','')
youtube_df.text = youtube_df.text.str.replace('[\s]','')
youtube_df.text = youtube_df.text.str.strip() # 앞뒤 공백 제거

youtube_df.text 

0       매우늘증시마감상황정리해보겠습니다증권팀지수이기나와있습니다네오늘대장주인삼성전가국내에서갤...
1       하이닉스가차세대메모리반도체사업확장에본격적로돌입했습니다청주에새로운생산라인을신설하는등적...
2       47월13일금일장마감42분전이군6명의전문가는어떤마감전략을제시할까한번화면로보시도록하죠...
3       소식입니다삼성전가또다시사상최대실적을경신했습니다갤럭시시리즈의위력을다시한번확인할수있었습...
4       삼성전가중국산시성시안시에서낸드플래시반도체공장기공식을열었습니다삼성전가반도체사업을시작한...
                              ...                        
6358    네방금전의말씀을드렸습니다만지금주가지수가급락한가장큰이유는바로삼성전와하이닉스에대한외국인...
6359    4시장의유망기업을살펴보고투포인트까지알가는시간입니다기업탐방업회사다녀왔습니다놈이니소치페...
6360    4긴급시장입니다방금전에시장에대해서전반적인걸스학교봐때문에이전에여러분들이정말궁금해하실어...
6361    투를하는사람들의깊이있는대화3%저녁라이브9월29일순서시작하겠습니다반갑습니다점프로네안녕...
6362    4에온제가깜빡하고많이파토란샴푸를안썼더니머리가좀많이망가진것같습니다여러분의멋있는데스타일...
Name: text, Length: 6363, dtype: object

In [18]:
youtube_df['token'] = ''

In [25]:
start = (youtube_df['token'] != '').sum()

In [ ]:
# 토큰화
for i in range(start,len(youtube_df)):
    temp = []
    
    temp = okt.nouns(youtube_df['text'].iloc[i])
    temp = [word for word in temp if not word in stopwords] # 불용어 제거
    temp = [word for word in temp if len(word) > 1] # 불용어 제거
#     print(i,'번 토큰화, 불용어 제거 완료')
    
    temp = list(set(temp))

    youtube_df['token'].iloc[i] = temp

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
youtube_df.to_csv('youtube_token.csv', index=False)